### response based on the prompt

In [101]:
from langchain.embeddings.base import Embeddings
from langchain.llms.base import LLM
from typing import List, Dict, Any
import requests
import numpy as np
import faiss
import os
from dotenv import load_dotenv





#### euri emdeddings with euri api key (generates the embeddings)

In [115]:
import requests
import numpy as np

def generate_embeddings(text: str) :
    url = "https://api.euron.one/api/v1/euri/alpha/embeddings"
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer "enter_api_key_here_itself"
    }
    payload = {
        "input":text,
        "model": "text-embedding-3-small"
    }

    response = requests.post(url, headers=headers, json=payload)
    data = response.json()
    
    # Convert to numpy array for vector operations
    embedding = np.array(data['data'][0]['embedding'])
    
    return embedding

#### euri chat model (gives the response based on the prompt)

In [116]:

from euriai import EuriaiClient
def generate_response(prompt):
    client = EuriaiClient(
        api_key="eenter_api_key_here_itself",
        model="gpt-4.1-nano"  
    )

    response = client.generate_completion(
        prompt=prompt,
        temperature=0,
        max_tokens=300
    )

    print(response)
    return response['choices'][0]["message"]["content"]

#### reading my own knowledge base  (using my own sample/private file )

In [117]:
from langchain.document_loaders import TextLoader

document = TextLoader("abhishek.txt").load()
document


#alterntaive option to read file

# with open ("abhishek.txt") as file:
#     document = file.read()
#     print(document)

[Document(metadata={'source': 'abhishek.txt'}, page_content='Transforming Healthcare for Millions\nDr. Abhihek vision for affordable healthcare wasn’t just a business plan—it was her lifelong passion. Over the past decade, her company, MediBridge, has impacted over 2 million patients across 25 countries, bringing essential healthcare services and digital consultations to underserved communities. Many of these patients, like Priya herself in her youth, faced barriers to quality care. MediBridge became their bridge to better health and hope for a brighter future.\n\nIn 2023, MediBridge was acquired by HealthNext in a landmark deal valued at ₹300 crore. Despite the acquisition, Dr. Priya stayed committed to her cause. MediBridge continued to offer some of the most affordable telemedicine and diagnostics packages in the industry.\n\nThe Doctor and Innovator: Abhishek Dual Impact\nAbhishek journey is not just one of business success; it’s a testament to her dedication as a physician. Throug

#### converting the above text into chunks/documents

In [118]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)
docs = splitter.split_documents(document)
docs

[Document(metadata={'source': 'abhishek.txt'}, page_content='Transforming Healthcare for Millions'),
 Document(metadata={'source': 'abhishek.txt'}, page_content='Dr. Abhihek vision for affordable healthcare wasn’t just a business plan—it was her lifelong passion. Over the past decade, her company, MediBridge, has impacted over 2 million patients across 25 countries, bringing essential healthcare services and digital consultations to underserved communities.'),
 Document(metadata={'source': 'abhishek.txt'}, page_content='to underserved communities. Many of these patients, like Priya herself in her youth, faced barriers to quality care. MediBridge became their bridge to better health and hope for a brighter future.'),
 Document(metadata={'source': 'abhishek.txt'}, page_content='In 2023, MediBridge was acquired by HealthNext in a landmark deal valued at ₹300 crore. Despite the acquisition, Dr. Priya stayed committed to her cause. MediBridge continued to offer some of the most affordable t

In [119]:
len(docs)

21

In [120]:
lengths = []  

for doc in docs:
    content_length = len(doc.page_content) 
    lengths.append(content_length)         

print(lengths)  

[36, 299, 196, 266, 46, 295, 97, 261, 125, 38, 295, 54, 214, 261, 215, 293, 35, 226, 48, 298, 53]


In [121]:
docs[0].page_content

'Transforming Healthcare for Millions'

In [122]:
len(docs[0].page_content)

36

In [123]:
docs[20].page_content


'medical technology affordable and accessible for all.'

#### converting the entire documents into one single list for the embedding purpose

In [124]:
texts = [doc.page_content for doc in docs]
texts

['Transforming Healthcare for Millions',
 'Dr. Abhihek vision for affordable healthcare wasn’t just a business plan—it was her lifelong passion. Over the past decade, her company, MediBridge, has impacted over 2 million patients across 25 countries, bringing essential healthcare services and digital consultations to underserved communities.',
 'to underserved communities. Many of these patients, like Priya herself in her youth, faced barriers to quality care. MediBridge became their bridge to better health and hope for a brighter future.',
 'In 2023, MediBridge was acquired by HealthNext in a landmark deal valued at ₹300 crore. Despite the acquisition, Dr. Priya stayed committed to her cause. MediBridge continued to offer some of the most affordable telemedicine and diagnostics packages in the industry.',
 'The Doctor and Innovator: Abhishek Dual Impact',
 'Abhishek journey is not just one of business success; it’s a testament to her dedication as a physician. Throughout her career, he

### converting into embeddings

In [125]:
embeddings = [generate_embeddings(i) for i in texts]

In [126]:
generate_embeddings("my name is abhishek")


array([ 0.01690955, -0.02679409, -0.00042851, ...,  0.0048325 ,
        0.02119885,  0.00627635])

In [127]:
generate_embeddings("This is a test input.")


array([ 0.02032711,  0.01127243,  0.01497341, ...,  0.01415411,
       -0.01671089, -0.00123601])

## creating embeddings

#### -> The code processes the first 8 texts from a list.
#### -> It skips empty or whitespace-only texts.
#### -> For each valid text, it generates an embedding and stores both the embedding and the original text.
#### -> It provides progress updates with partial text and embedding information.


In [129]:
embeddings  = []
valid_texts = []

for i ,text in enumerate(texts[0:8]):
    clean_text = text.strip()   #cleaning the text
    if not clean_text:    # checking if the text is empty
        print(f"Skipping empty text at index {i}")
        continue
    embedding = generate_embeddings(text)  #this is where the embeddings are generated
    embeddings.append(embedding)
    valid_texts.append(text)
    print(f"Processed {i+1}/{len(texts)}: {text[:30]}... -> {embedding[:5]}")  # Print first 30 characters of text and first 5 elements of embedding

Processed 1/21: Transforming Healthcare for Mi... -> [ 0.01975284  0.00706196  0.06248612  0.05396529 -0.00056079]
Processed 2/21: Dr. Abhihek vision for afforda... -> [ 0.04272165 -0.02653647  0.06613466  0.05952636 -0.01937317]
Processed 3/21: to underserved communities. Ma... -> [ 0.0399706  -0.02989491  0.03689807  0.09455648 -0.00435275]
Processed 4/21: In 2023, MediBridge was acquir... -> [-0.00016461 -0.05070317  0.05668828  0.03727315  0.00365554]
Processed 5/21: The Doctor and Innovator: Abhi... -> [ 0.05285941 -0.0049322   0.01850229  0.03784219 -0.00155183]
Processed 6/21: Abhishek journey is not just o... -> [ 0.03138234 -0.01424949  0.06477275  0.0695612  -0.00469191]
Processed 7/21: or leading community health ca... -> [ 0.02485341 -0.01517221  0.06724305  0.05653619 -0.01015025]
Processed 8/21: Her relentless pursuit of affo... -> [ 0.05266743 -0.05771006  0.03395368  0.09261624  0.01680876]


In [130]:
embeddings

[array([ 0.01975284,  0.00706196,  0.06248612, ...,  0.0012168 ,
        -0.0383954 ,  0.00827554]),
 array([ 0.04272165, -0.02653647,  0.06613466, ...,  0.02094781,
         0.00125116,  0.01863748]),
 array([ 0.0399706 , -0.02989491,  0.03689807, ...,  0.03487739,
        -0.01369491, -0.00535271]),
 array([-0.00016461, -0.05070317,  0.05668828, ...,  0.01407475,
         0.02353902,  0.01184858]),
 array([ 0.05285941, -0.0049322 ,  0.01850229, ..., -0.04810296,
         0.01236976,  0.01800869]),
 array([ 0.03138234, -0.01424949,  0.06477275, ...,  0.00106839,
        -0.00430252,  0.02212726]),
 array([ 0.02485341, -0.01517221,  0.06724305, ...,  0.02379023,
        -0.0075048 ,  0.0479557 ]),
 array([ 0.05266743, -0.05771006,  0.03395368, ...,  0.01603835,
         0.00143225, -0.01122685])]

In [131]:
embeddings[0]

array([ 0.01975284,  0.00706196,  0.06248612, ...,  0.0012168 ,
       -0.0383954 ,  0.00827554])

In [132]:
docs[0:10]

[Document(metadata={'source': 'abhishek.txt'}, page_content='Transforming Healthcare for Millions'),
 Document(metadata={'source': 'abhishek.txt'}, page_content='Dr. Abhihek vision for affordable healthcare wasn’t just a business plan—it was her lifelong passion. Over the past decade, her company, MediBridge, has impacted over 2 million patients across 25 countries, bringing essential healthcare services and digital consultations to underserved communities.'),
 Document(metadata={'source': 'abhishek.txt'}, page_content='to underserved communities. Many of these patients, like Priya herself in her youth, faced barriers to quality care. MediBridge became their bridge to better health and hope for a brighter future.'),
 Document(metadata={'source': 'abhishek.txt'}, page_content='In 2023, MediBridge was acquired by HealthNext in a landmark deal valued at ₹300 crore. Despite the acquisition, Dr. Priya stayed committed to her cause. MediBridge continued to offer some of the most affordable t

### storing embeddings into vector database

In [145]:
import faiss
from langchain.vectorstores.faiss import FAISS
from langchain.docstore.document import Document


dimension = embeddings[0].shape[0]


faiss_index = faiss.IndexFlatL2(dimension)
faiss_index.add(np.array(embeddings).astype('float32'))

In [146]:
generate_response("tell me about abhishek")

{'id': 'chatcmpl-Bhxe0liICoCZAvJh1KyOjOgcv4LuP', 'object': 'chat.completion', 'created': 1749817076, 'model': 'gpt-4.1-nano', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'Certainly! "Abhishek" is a common male given name in India and other parts of South Asia. It is derived from Sanskrit, meaning "anointing" or "consecration," often associated with religious rituals and ceremonies where a deity or person is anointed with sacred substances. The name symbolizes purity, blessing, and auspiciousness.\n\nMany individuals named Abhishek have achieved recognition in various fields such as entertainment, sports, politics, and business. If you are referring to a specific person named Abhishek, please provide more context or details so I can offer more targeted information.'}, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 13, 'completion_tokens': 122, 'total_tokens': 135}}


'Certainly! "Abhishek" is a common male given name in India and other parts of South Asia. It is derived from Sanskrit, meaning "anointing" or "consecration," often associated with religious rituals and ceremonies where a deity or person is anointed with sacred substances. The name symbolizes purity, blessing, and auspiciousness.\n\nMany individuals named Abhishek have achieved recognition in various fields such as entertainment, sports, politics, and business. If you are referring to a specific person named Abhishek, please provide more context or details so I can offer more targeted information.'

In [138]:
document = [Document(page_content=text) for text in texts[:len(embeddings)]]

In [139]:
document

[Document(metadata={}, page_content='Transforming Healthcare for Millions'),
 Document(metadata={}, page_content='Dr. Abhihek vision for affordable healthcare wasn’t just a business plan—it was her lifelong passion. Over the past decade, her company, MediBridge, has impacted over 2 million patients across 25 countries, bringing essential healthcare services and digital consultations to underserved communities.'),
 Document(metadata={}, page_content='to underserved communities. Many of these patients, like Priya herself in her youth, faced barriers to quality care. MediBridge became their bridge to better health and hope for a brighter future.'),
 Document(metadata={}, page_content='In 2023, MediBridge was acquired by HealthNext in a landmark deal valued at ₹300 crore. Despite the acquisition, Dr. Priya stayed committed to her cause. MediBridge continued to offer some of the most affordable telemedicine and diagnostics packages in the industry.'),
 Document(metadata={}, page_content='Th

In [140]:
embeddings

[array([ 0.01975284,  0.00706196,  0.06248612, ...,  0.0012168 ,
        -0.0383954 ,  0.00827554]),
 array([ 0.04272165, -0.02653647,  0.06613466, ...,  0.02094781,
         0.00125116,  0.01863748]),
 array([ 0.0399706 , -0.02989491,  0.03689807, ...,  0.03487739,
        -0.01369491, -0.00535271]),
 array([-0.00016461, -0.05070317,  0.05668828, ...,  0.01407475,
         0.02353902,  0.01184858]),
 array([ 0.05285941, -0.0049322 ,  0.01850229, ..., -0.04810296,
         0.01236976,  0.01800869]),
 array([ 0.03138234, -0.01424949,  0.06477275, ...,  0.00106839,
        -0.00430252,  0.02212726]),
 array([ 0.02485341, -0.01517221,  0.06724305, ...,  0.02379023,
        -0.0075048 ,  0.0479557 ]),
 array([ 0.05266743, -0.05771006,  0.03395368, ...,  0.01603835,
         0.00143225, -0.01122685])]

### creates a vector store using FAISS (a similarity search library) integrated with LangChain, to store and search document embeddings

In [149]:
from langchain.vectorstores.faiss import FAISS
from langchain.docstore.document import Document

document = [Document(page_content=text) for text in texts[:len(embeddings)]]  # Creating Document objects

import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

faiss_index = faiss.IndexFlatL2(dimension)  #Initializing FAISS index

# Creating a FAISS vector store
vector_store = FAISS(
    embedding_function= generate_embeddings,
    index= faiss_index,
    docstore= InMemoryDocstore(),  #memory store to keep track of documents
    index_to_docstore_id={},  # A mapping between FAISS index IDs and document IDs 
)

vector_store.add_documents(document)   # Adding documents to the vector store

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


['3e3fd279-ba5d-4485-95f0-907c1d17cf99',
 'a3d3e10d-0c5b-450f-9778-916601bde413',
 'bfdd53d8-d087-4efe-8297-3fc8ea197b5e',
 'eb7782ca-a180-48f4-a266-9cf4b45cdefe',
 'aeabf861-c84e-4fb1-b269-c145caffc252',
 '6be5aad3-1667-4828-8cb9-55c662eb60a4',
 '1b21b479-aef5-407c-b103-df1cf84151f1',
 'f6f87c44-071a-4d50-9459-b401e1a1c1b9']

In [150]:
vector_store.save_local("faiss_index")

In [152]:
retriver = vector_store.as_retriever(search_kwargs={"k": 2})  # Retrieving top 2 documents

### real use case of langchain


In [159]:
from euriai import EuriaiLangChainLLM
from dotenv import load_dotenv
import os

load_dotenv() 

llm = EuriaiLangChainLLM(
    api_key = os.getenv("EURI_API_KEY"),
    model="gpt-4.1-nano",
    temperature=0,
    max_tokens=300
)

print(llm.invoke("Write a poem about time travel."))

In whispers of the ticking clock’s embrace,  
A journey threads through time’s uncharted space,  
Where past and future dance in silent grace,  
And moments fold within a fleeting trace.

A whisper of the bygone days gone by,  
Echoes of laughter, tears that once did flow,  
A glimpse of dreams that in the shadows lie,  
Reminding us of all we used to know.

Through swirling mists of what might yet be,  
We chase the echoes of eternity,  
A fleeting glimpse of what’s to come,  
A fleeting beat of time’s relentless drum.

Yet in each tick, a mystery unfurls,  
A chance to change the course of worlds,  
To hold the past, embrace the now,  
And shape the future’s silent vow.

Time travel’s art, both wondrous and wise,  
A voyage through the endless skies,  
Where moments merge and stories blend,  
A dance with time—no beginning, no end.


In [161]:
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever = retriver)

In [162]:
rag_chain("who is abhishek")

/var/folders/dp/w9766y9x00j7vmxb8b37p1bc0000gp/T/ipykernel_91481/748377218.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  rag_chain("who is abhishek")


{'query': 'who is abhishek',
 'result': 'Based on the provided context, Abhishek is a physician who is dedicated to making healthcare accessible. She has been actively involved in developing AI-powered diagnostic tools and leading community health camps to simplify healthcare for everyone.'}

In [164]:
rag_chain("who is arun")

{'query': 'who is arun',
 'result': "I don't have information about who Arun is based on the provided context."}

In [165]:
generate_response("who is abhishek")

{'id': 'chatcmpl-BhxpqMJAZGUDmbL4hDrrjGwPjRGRt', 'object': 'chat.completion', 'created': 1749817810, 'model': 'gpt-4.1-nano', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '"Abhishek" is a common given name in India and other South Asian countries, typically used for males. It means "anointed" or "ritually blessed" in Sanskrit. Without additional context, it\'s difficult to identify a specific individual named Abhishek. Could you please provide more details or specify which Abhishek you are referring to?'}, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 12, 'completion_tokens': 71, 'total_tokens': 83}}


'"Abhishek" is a common given name in India and other South Asian countries, typically used for males. It means "anointed" or "ritually blessed" in Sanskrit. Without additional context, it\'s difficult to identify a specific individual named Abhishek. Could you please provide more details or specify which Abhishek you are referring to?'

In [166]:
generate_response("who is arun")

{'id': 'chatcmpl-Bhxpz9a4CIsQFElaZfUC2445qqfyK', 'object': 'chat.completion', 'created': 1749817819, 'model': 'gpt-4.1-nano', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '"Arun" is a common given name in several cultures, particularly in India, where it is often used for males and means "sun" or "dawn" in Sanskrit. Without additional context, it\'s difficult to identify a specific individual named Arun. Could you please provide more details or specify which Arun you are referring to?'}, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 11, 'completion_tokens': 66, 'total_tokens': 77}}


'"Arun" is a common given name in several cultures, particularly in India, where it is often used for males and means "sun" or "dawn" in Sanskrit. Without additional context, it\'s difficult to identify a specific individual named Arun. Could you please provide more details or specify which Arun you are referring to?'

In [ ]:
rag_chain("give me the comapny whre abhishek is working")

In [ ]:
rag_chain("what is the name of the company whre abhishek is working")

In [ ]:
rag_chain("currently where abhishek is working")